In [5]:
from transformers import pipeline

In [47]:
import pandas as pd
import numpy as np

In [ ]:
dataset = pd.read_csv("modified_dataset.csv")

In [9]:
dataset.head(2)

,Unnamed: 0,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Director,Star1,Star2,Star3,Star4,movie_age,Hybrid_Score,Popularity_Score,Overview_length,combined_Overview
0,0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,31,85.2,0.612103,19,Two imprisoned men bond over a number of years...
1,1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,53,96.8,0.470561,16,An organized crime dynasty's aging patriarch t...


In [10]:
dataset.drop("Unnamed: 0",axis=1,inplace=True)

#### Mapping the labels to 5 categories

In [268]:
def map_certificate_to_group(certificate):
    # Define our groups
    general_family = ['G', 'GP', 'U', 'UA', 'U/A']
    parental_guidance = ['PG-13', "PG"]
    teen_movies = ["TV-14","16"]
    mature = ['TV-MA', 'R', 'A']
    special_cases = ['Approved', 'Unrated', 'Passed']
    
    if certificate in general_family:

        return 'G - Suitable for all ages'
    
    elif certificate in parental_guidance:

        return 'PG - Parental Guidance'
    
    elif certificate in teen_movies:

        return 'TV - Teen'
    
    elif certificate in mature:
        
        return 'A - Adult'
    
    elif certificate in special_cases:

        return 'nan'
    

# Apply to your dataset
dataset['Certificate_Group'] = dataset['Certificate'].map(map_certificate_to_group)

In [269]:
dataset['Certificate_Group'].unique()

array(['A - Adult', 'G - Suitable for all ages', 'PG - Parental Guidance',
       None, 'nan', 'TV - Teen'], dtype=object)

In [270]:
dataset['Certificate_Group'].fillna('nan',inplace=True)

C:\Users\amitt\AppData\Local\Temp\ipykernel_54156\2929904118.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['Certificate_Group'].fillna('nan',inplace=True)


In [271]:
dataset['Certificate_Group'].unique()

array(['A - Adult', 'G - Suitable for all ages', 'PG - Parental Guidance',
       'nan', 'TV - Teen'], dtype=object)

In [272]:
dataset['Certificate_Group'].value_counts()

Certificate_Group
G - Suitable for all ages    424
A - Adult                    344
nan                          150
PG - Parental Guidance        80
TV - Teen                      2
Name: count, dtype: int64

In [273]:
candidate_labels = list(dataset["Certificate_Group"].unique())

In [274]:
candidate_labels.remove("nan")

In [275]:
candidate_labels

['A - Adult',
 'G - Suitable for all ages',
 'PG - Parental Guidance',
 'TV - Teen']

In [276]:
classifier = pipeline("zero-shot-classification", 
                     model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")

Device set to use cpu


In [277]:
def generate_predictions_categories(sequence,categories):

    predictions = classifier(sequence, candidate_labels=categories)

    index = np.argmax(predictions["scores"])

    label = predictions["labels"][index]

    return label

In [278]:
generate_predictions_categories(dataset["combined_Overview"][300],candidate_labels)

'A - Adult'

In [279]:
mask = (dataset['Certificate_Group'].astype(str) == "nan")

In [280]:
mask

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997     True
998     True
999     True
Name: Certificate_Group, Length: 1000, dtype: bool

In [281]:
dataset['Certificate_Group'] 

0                      A - Adult
1                      A - Adult
2      G - Suitable for all ages
3                      A - Adult
4      G - Suitable for all ages
                 ...            
995                    A - Adult
996    G - Suitable for all ages
997                          nan
998                          nan
999                          nan
Name: Certificate_Group, Length: 1000, dtype: object

In [282]:
dataset.loc[mask,'combined_Overview']

30     When a ronin requesting seppuku at a feudal lo...
54     In 1950, amid-st the ravages of the Korean War...
77     An executive of a shoe company becomes a victi...
82     A screenwriter develops a dangerous relationsh...
83     Dictator Adenoid Hynkel tries to expand his em...
                             ...                        
926    A disaffected soldier returns to his hometown ...
944    In the future, the Japanese government capture...
997    In Hawaii in 1941, a private is cruelly punish...
998    Several survivors of a torpedoed merchant ship...
999    A man in London tries to help a counter-espion...
Name: combined_Overview, Length: 150, dtype: object

#### Mapping the categories for the Certificate Group

In [283]:
dataset.loc[mask, 'Certificate_Group'] = dataset.loc[mask, 'combined_Overview'].apply(
    lambda x: generate_predictions_categories(x, candidate_labels)
)

In [297]:
dataset['Certificate_Group'].value_counts()

Certificate_Group
A - Adult                    472
G - Suitable for all ages    431
PG - Parental Guidance        86
TV - Teen                     11
Name: count, dtype: int64

In [301]:
dataset.drop(["Certificate","Predicted_Rating"],axis=1,inplace=True)

In [305]:
dataset.to_csv("modified_dataset_with_certificates.csv",index=False)

In [304]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import pandas as pd

In [306]:
new_dataset = pd.read_csv("modified_dataset_with_certificates.csv")

In [343]:
from dotenv import load_dotenv

load_dotenv()

True

In [329]:
new_dataset["index"] = new_dataset.index 

In [330]:
new_dataset["index"] = new_dataset["index"] + 100

In [332]:
new_dataset["Tagged description"] = new_dataset["index"].astype("str") + " " + new_dataset["combined_Overview"] 

In [335]:
new_dataset["Tagged description"].to_csv("tagged_description.txt",
                                     sep = "\n",
                                     index=False,
                                     header = False)

In [336]:
loader = TextLoader("tagged_description.txt", encoding="utf-8")

In [337]:
raw_documents = loader.load()

In [339]:
text_splitter = CharacterTextSplitter(chunk_size=0,chunk_overlap=0,separator="\n")

In [340]:
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 128, which is longer than the specified 0
Created a chunk of size 127, which is longer than the specified 0
Created a chunk of size 214, which is longer than the specified 0
Created a chunk of size 181, which is longer than the specified 0
Created a chunk of size 130, which is longer than the specified 0
Created a chunk of size 175, which is longer than the specified 0
Created a chunk of size 160, which is longer than the specified 0
Created a chunk of size 208, which is longer than the specified 0
Created a chunk of size 182, which is longer than the specified 0
Created a chunk of size 143, which is longer than the specified 0
Created a chunk of size 181, which is longer than the specified 0
Created a chunk of size 254, which is longer than the specified 0
Created a chunk of size 150, which is longer than the specified 0
Created a chunk of size 207, which is longer than the specified 0
Created a chunk of size 207, which is longer than the specified 0
Created a 

In [ ]:
db_movies = Chroma.from_documents(documents,embedding=OpenAIEmbeddings())

In [ ]:
query = "A movie on spiderman"

docs_obtained = db_movies.similarity_search(query,k=10)

In [346]:
docs_obtained

[Document(id='fdd6bfaf-7725-4246-9cbb-ef17df746607', metadata={'source': 'tagged_description.txt'}, page_content="145 A filmmaker recalls his childhood when falling in love with the pictures at the cinema of his home village and forms a deep friendship with the cinema's projectionist. Drama, Romance"),
 Document(id='11a6e6e9-1cb7-4fd2-beab-78b3a4841aa3', metadata={'source': 'tagged_description.txt'}, page_content="854 An offbeat romantic comedy about a woman who doesn't believe true love exists, and the young man who falls for her. Comedy, Drama, Romance"),
 Document(id='b4fba517-3d99-483c-9d77-f1ab68311e95', metadata={'source': 'tagged_description.txt'}, page_content='806 Decades of a love triangle concerning two friends and an impulsive woman. Drama, Romance'),
 Document(id='16d44122-0920-4934-9d82-c740a67ca4ca', metadata={'source': 'tagged_description.txt'}, page_content='618 A love story between a girl who loves reading books, and a boy who has previously checked out all of the lib

In [355]:
new_dataset[new_dataset["index"] == int(docs_obtained[8].page_content.split(" ")[0].strip())]

,Poster_Link,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Director,Star1,Star2,Star3,Star4,movie_age,Hybrid_Score,Popularity_Score,Overview_length,combined_Overview,Certificate_Group,index,Tagged description
859,https://m.media-amazon.com/images/M/MV5BZTU5ZT...,Repulsion,1965,105 min,"Drama, Horror, Thriller",7.7,A sex-repulsed woman who disapproves of her si...,Roman Polanski,Catherine Deneuve,Ian Hendry,John Fraser,Yvonne Furneaux,60,85.4,0.006159,20,A sex-repulsed woman who disapproves of her si...,A - Adult,959,959 A sex-repulsed woman who disapproves of he...


In [ ]:
# def retreive_semantice_recommendations(query:str, top_k:int) -> pd.DataFrame:

#         recs = db_movies.similarity_search(query,k=top_k)

#         movies_list = []

#         for rec in recs:

#             # '"' helps split out the quotation marks
#             movies_list.append(int(rec.page_content.strip('"').split(" ")[0]))

#         retrieved_set = new_dataset[new_dataset["index"].isin(movies_list)]

#         return retrieved_set

# retreive_semantice_recommendations("spiderman",5)


,Poster_Link,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Director,Star1,Star2,Star3,Star4,movie_age,Hybrid_Score,Popularity_Score,Overview_length,combined_Overview,Certificate_Group,index,Tagged description
58,https://m.media-amazon.com/images/M/MV5BMjMwND...,Spider-Man: Into the Spider-Verse,2018,117 min,"Animation, Action, Adventure",8.4,Teen Miles Morales becomes the Spider-Man of h...,Bob Persichetti,Peter Ramsey,Rodney Rothman,Shameik Moore,Jake Johnson,7,85.8,0.171842,26,Teen Miles Morales becomes the Spider-Man of h...,G - Suitable for all ages,158,158 Teen Miles Morales becomes the Spider-Man ...
303,https://m.media-amazon.com/images/M/MV5BNGYxZj...,Kumonosu-jô,1957,110 min,"Drama, History",8.1,"A war-hardened general, egged on by his ambiti...",Akira Kurosawa,Toshirô Mifune,Minoru Chiaki,Isuzu Yamada,Takashi Shimura,68,81.0,0.005588,23,"A war-hardened general, egged on by his ambiti...",A - Adult,403,"403 A war-hardened general, egged on by his am..."
502,https://m.media-amazon.com/images/M/MV5BMTczNT...,Iron Man,2008,126 min,"Action, Adventure, Sci-Fi",7.9,"After being held captive in an Afghan cave, bi...",Jon Favreau,Robert Downey Jr.,Gwyneth Paltrow,Terrence Howard,Jeff Bridges,17,79.0,0.372702,22,"After being held captive in an Afghan cave, bi...",G - Suitable for all ages,602,602 After being held captive in an Afghan cave...
733,https://m.media-amazon.com/images/M/MV5BODAzND...,Birdman or (The Unexpected Virtue of Ignorance),2014,119 min,"Comedy, Drama",7.7,A washed-up superhero actor attempts to revive...,Alejandro G. Iñárritu,Michael Keaton,Zach Galifianakis,Edward Norton,Andrea Riseborough,11,83.0,0.161791,19,A washed-up superhero actor attempts to revive...,A - Adult,833,833 A washed-up superhero actor attempts to re...
908,https://m.media-amazon.com/images/M/MV5BMTMzNz...,Kick-Ass,2010,117 min,"Action, Comedy, Crime",7.6,Dave Lizewski is an unnoticed high school stud...,Matthew Vaughn,Aaron Taylor-Johnson,Nicolas Cage,Chloë Grace Moretz,Garrett M. Brown,15,70.0,0.149689,33,Dave Lizewski is an unnoticed high school stud...,G - Suitable for all ages,1008,1008 Dave Lizewski is an unnoticed high school...


In [365]:
def retrieve_semantic_recommendations(query: str, top_k: int) -> pd.DataFrame:
    # Get semantic search results
    recs = db_movies.similarity_search(query, k=top_k)
    
    movies_list = []
    similarity_scores = []
    
    for rec in recs:
        # Extract movie ID and similarity score
        content = rec.page_content.strip('"')
        movies_list.append(int(content.split(" ")[0]))
        similarity_scores.append(rec.metadata.get('score', 0))  # Get similarity score
    
    # Get matching movies from dataset
    retrieved_set = new_dataset[new_dataset["index"].isin(movies_list)].copy()
    
    # Add similarity scores and combine with Hybrid_Score
    retrieved_set['Similarity_Score'] = retrieved_set['index'].map(
        dict(zip(movies_list, similarity_scores))
    )
    
    # Normalize scores to same scale (0-100) if needed
    retrieved_set['Similarity_Score'] = retrieved_set['Similarity_Score'] * 100
    
    # Combine scores (adjust weights as needed)
    retrieved_set['Combined_Score'] = (
        0.5 * retrieved_set['Hybrid_Score'] + 
        0.5 * retrieved_set['Similarity_Score']
    )
    
    # Sort by combined score
    return retrieved_set.sort_values('Combined_Score', ascending=False).head(top_k)

In [367]:
retrieve_semantic_recommendations("spiderman",10)

,Poster_Link,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Director,Star1,Star2,...,movie_age,Hybrid_Score,Popularity_Score,Overview_length,combined_Overview,Certificate_Group,index,Tagged description,Similarity_Score,Combined_Score
378,https://m.media-amazon.com/images/M/MV5BMTY5OT...,The Incredibles,2004,115 min,"Animation, Action, Adventure",8.0,"A family of undercover superheroes, while tryi...",Brad Bird,Craig T. Nelson,Samuel L. Jackson,...,21,86.0,0.275225,21,"A family of undercover superheroes, while tryi...",G - Suitable for all ages,478,"478 A family of undercover superheroes, while ...",0,43.0
58,https://m.media-amazon.com/images/M/MV5BMjMwND...,Spider-Man: Into the Spider-Verse,2018,117 min,"Animation, Action, Adventure",8.4,Teen Miles Morales becomes the Spider-Man of h...,Bob Persichetti,Peter Ramsey,Rodney Rothman,...,7,85.8,0.171842,26,Teen Miles Morales becomes the Spider-Man of h...,G - Suitable for all ages,158,158 Teen Miles Morales becomes the Spider-Man ...,0,42.9
733,https://m.media-amazon.com/images/M/MV5BODAzND...,Birdman or (The Unexpected Virtue of Ignorance),2014,119 min,"Comedy, Drama",7.7,A washed-up superhero actor attempts to revive...,Alejandro G. Iñárritu,Michael Keaton,Zach Galifianakis,...,11,83.0,0.161791,19,A washed-up superhero actor attempts to revive...,A - Adult,833,833 A washed-up superhero actor attempts to re...,0,41.5
303,https://m.media-amazon.com/images/M/MV5BNGYxZj...,Kumonosu-jô,1957,110 min,"Drama, History",8.1,"A war-hardened general, egged on by his ambiti...",Akira Kurosawa,Toshirô Mifune,Minoru Chiaki,...,68,81.0,0.005588,23,"A war-hardened general, egged on by his ambiti...",A - Adult,403,"403 A war-hardened general, egged on by his am...",0,40.5
502,https://m.media-amazon.com/images/M/MV5BMTczNT...,Iron Man,2008,126 min,"Action, Adventure, Sci-Fi",7.9,"After being held captive in an Afghan cave, bi...",Jon Favreau,Robert Downey Jr.,Gwyneth Paltrow,...,17,79.0,0.372702,22,"After being held captive in an Afghan cave, bi...",G - Suitable for all ages,602,602 After being held captive in an Afghan cave...,0,39.5
60,https://m.media-amazon.com/images/M/MV5BMjMxNj...,Avengers: Infinity War,2018,149 min,"Action, Adventure, Sci-Fi",8.4,The Avengers and their allies must be willing ...,Anthony Russo,Joe Russo,Robert Downey Jr.,...,7,74.4,0.499390,32,The Avengers and their allies must be willing ...,G - Suitable for all ages,160,160 The Avengers and their allies must be will...,0,37.2
737,https://m.media-amazon.com/images/M/MV5BMzA2ND...,Captain America: The Winter Soldier,2014,136 min,"Action, Adventure, Sci-Fi",7.7,As Steve Rogers struggles to embrace his role ...,Anthony Russo,Joe Russo,Chris Evans,...,11,72.8,0.294993,38,As Steve Rogers struggles to embrace his role ...,G - Suitable for all ages,837,837 As Steve Rogers struggles to embrace his r...,0,36.4
497,https://m.media-amazon.com/images/M/MV5BMjAzOD...,The Fall,2006,117 min,"Adventure, Drama, Fantasy",7.9,In a hospital on the outskirts of 1920s Los An...,Tarsem Singh,Lee Pace,Catinca Untaru,...,19,70.0,0.022251,55,In a hospital on the outskirts of 1920s Los An...,A - Adult,597,597 In a hospital on the outskirts of 1920s Lo...,0,35.0
908,https://m.media-amazon.com/images/M/MV5BMTMzNz...,Kick-Ass,2010,117 min,"Action, Comedy, Crime",7.6,Dave Lizewski is an unnoticed high school stud...,Matthew Vaughn,Aaron Taylor-Johnson,Nicolas Cage,...,15,70.0,0.149689,33,Dave Lizewski is an unnoticed high school stud...,G - Suitable for all ages,1008,1008 Dave Lizewski is an unnoticed high school...,0,35.0
749,https://m.media-amazon.com/images/M/MV5BMTg5OT...,X: First Class,2011,131 min,"Action, Adventure, Sci-Fi",7.7,"In the 1960s, superpowered humans Charles Xavi...",Matthew Vaughn,James McAvoy,Michael Fassbender,...,14,69.8,0.223115,35,"In the 1960s, superpowered humans Charles Xavi...",G - Suitable for all ages,849,"849 In the 1960s, superpowered humans Charles ...",0,34.9


In [372]:
new_dataset["Released_Year"].min()

np.int64(1920)

In [379]:
def year_detrmine(year):

    if year >= 1920 and year<1930:

        return "1920s"
    
    elif year >= 1930 and year<1940:    

        return "1930s"
    elif year >= 1940 and year<1950:

        return "1940s"
    
    elif year >= 1950 and year<1960:

        return "1950s"
    
    elif year >= 1960 and year<1970:

        return "1960s"
    
    elif year >= 1970 and year<1980:

        return "1970s"
    
    elif year >= 1980 and year<1990:

        return "1980s"
    
    elif year >= 1990 and year<2000:


        return "1990s"
    
    elif year >= 2000 and year<2010:

        return "2000s"
    
    elif year >= 2010 and year<2020:

        return "2010s"
    
    elif year >= 2020:

        return "2020s"
    
    else:

        return "Unknown"





In [380]:
new_dataset["Year-range"] = new_dataset["Released_Year"].apply(year_detrmine)

In [381]:
new_dataset["Year-range"]

0      1990s
1      1970s
2      2000s
3      1970s
4      1950s
       ...  
995    1960s
996    1950s
997    1950s
998    1940s
999    1930s
Name: Year-range, Length: 1000, dtype: object

In [382]:
sorted(new_dataset["Year-range"].unique())

['1920s',
 '1930s',
 '1940s',
 '1950s',
 '1960s',
 '1970s',
 '1980s',
 '1990s',
 '2000s',
 '2010s',
 '2020s']

In [383]:
new_dataset.to_csv("modified_dataset_with_index.csv")